In [1]:
from pyvi import ViTokenizer, ViPosTagger
def remove_pattern(row_content):
    return re.sub(r'[?|$|.|!]',r'', row_content)
def tokenize(row_content):
    return ViTokenizer.tokenize(row_content)

In [2]:
import pandas
import re
df = pandas.read_csv('train.txt', sep='\t')
# print(df)
df["Content"] = df["Content"].apply(remove_pattern)
df["Content"]= df["Content"].apply(tokenize)
print(df)

      ID Sentiment                                            Content
0      0         1  Khung_cảnh đặc_trưng chưa bị pha_tạp Có nhiều ...
1      1         1  Tôi có dịp vào Gia_lai công_tác , sau ngày làm...
2      2         1  Biển đẹp , buổi tối ở bãi biển rất tấp_nập , q...
3      3         0           hải_sản ko phong_phú và chế_biến ko ngon
4      4         1  Tôi đến Hội_An khoảng 20 lần , lần nào cũng và...
5      5         1  Chùa trên núi cao , rất mát_mẻ Chùa quả là hoà...
6      6         1  Tôi có đến Dinh_III , ấn_tượng với các hiện_vậ...
7      7         1  Nếu mục_đích của bạn đến Bangkok là để mua_sắm...
8      9         1  Đây là 1 khách_sạn chứ không phải resort nhưng...
9     10         1  Nhà_thờ nằm ngay khu_vực trung_tâm , nhưng có ...
10    11         1  Chùa rộng , nằm trên đồi cao , tách_biệt khu d...
11    12         0  Bãi biển Cha_Am không phẳng , bãi cát không mị...
12    13         0  Ngay gần Rừng_Quốc_gia Ba Vì , không_gian rất ...
13    14         1  

In [3]:
def preprocessing(content):
    count_vect = CountVectorizer()
    X_counts = count_vect.fit_transform(content)
    tfidf_transformer = TfidfTransformer()
    X_tfidf = tfidf_transformer.fit_transform(X_counts)
    return X_tfidf

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib
import pickle

import pandas
import re

X_tfidf = preprocessing(df['Content'])
print(X_tfidf.shape)
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, df['Sentiment'], test_size=0.1, random_state=69)  
clf = GaussianNB().fit(X_train_tfidf.toarray(), y_train_tfidf)
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

(202, 2679)


In [28]:
print(X_test_tfidf.shape)
loaded_model = pickle.load(open("finalized_model.sav", 'rb'))
print(loaded_model)

y_pred = clf.predict(X_test_tfidf.toarray())

(21, 2679)
GaussianNB(priors=None, var_smoothing=1e-09)


In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
# average_precision = average_precision_score(test['Sentiment'], y_score)
print(f1_score(y_test_tfidf, y_pred, average="macro"))
print(precision_score(y_test_tfidf, y_pred, average="macro"))
print(recall_score(y_test_tfidf, y_pred, average="macro"))    

0.47500000000000003
0.475
0.475


In [30]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test_tfidf, y_pred))  

[[ 0  1]
 [ 1 19]]


In [35]:
import numpy as np
print(np.mean(y_pred == y_test_tfidf))  

0.9047619047619048
